## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import key

# Configure gmaps API key
gmaps.configure(api_key=key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/weather_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,48.99,90,0,3.00,clear sky
1,1,Cabo San Lucas,MX,22.8909,-109.9124,82.81,57,75,4.00,broken clouds
2,2,Severo-Kurilsk,RU,50.6789,156.1250,42.73,94,42,14.65,scattered clouds
3,3,Kapaa,US,22.0752,-159.3190,73.72,70,1,1.01,clear sky
4,4,Sao Filipe,CV,14.8961,-24.4956,77.61,75,100,4.76,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cabo San Lucas,MX,22.8909,-109.9124,82.81,57,75,4.00,broken clouds
3,3,Kapaa,US,22.0752,-159.3190,73.72,70,1,1.01,clear sky
4,4,Sao Filipe,CV,14.8961,-24.4956,77.61,75,100,4.76,overcast clouds
5,5,Rikitea,PF,-23.1203,-134.9692,73.65,74,1,4.94,clear sky
7,7,East London,ZA,-33.0153,27.9116,74.34,23,0,20.71,clear sky


In [5]:
preferred_cities_df.count()

City_ID                374
City                   374
Country                371
Lat                    374
Lng                    374
Max Temp               374
Humidity               374
Cloudiness             374
Wind Speed             374
Current Description    374
dtype: int64

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
preferred_cities_df.count()

City_ID                371
City                   371
Country                371
Lat                    371
Lng                    371
Max Temp               371
Humidity               371
Cloudiness             371
Wind Speed             371
Current Description    371
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cabo San Lucas,MX,82.81,broken clouds,22.8909,-109.9124,
3,Kapaa,US,73.72,clear sky,22.0752,-159.3190,
4,Sao Filipe,CV,77.61,overcast clouds,14.8961,-24.4956,
5,Rikitea,PF,73.65,clear sky,-23.1203,-134.9692,
7,East London,ZA,74.34,clear sky,-33.0153,27.9116,
8,Mayo,US,78.04,broken clouds,38.8876,-76.5119,
11,Torbay,CA,65.75,broken clouds,47.6666,-52.7314,
13,Naze,JP,76.50,overcast clouds,28.3667,129.4833,
15,Khani,GE,76.57,clear sky,41.9563,42.9566,
16,Bethel,US,73.49,overcast clouds,41.3712,-73.4140,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params)
    try:
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels.json()["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [17]:
hotel_df.count()

City                   371
Country                371
Max Temp               371
Current Description    371
Lat                    371
Lng                    371
Hotel Name             371
dtype: int64

In [42]:
hotel_df["Hotel Name"].isnull().sum()

0

In [43]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"].notna()]

In [46]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cabo San Lucas,MX,82.81,broken clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
3,Kapaa,US,73.72,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Sao Filipe,CV,77.61,overcast clouds,14.8961,-24.4956,Hotel Xaguate
5,Rikitea,PF,73.65,clear sky,-23.1203,-134.9692,People ThankYou
7,East London,ZA,74.34,clear sky,-33.0153,27.9116,Tu Casa
8,Mayo,US,78.04,broken clouds,38.8876,-76.5119,Resorts Resource Group International
11,Torbay,CA,65.75,broken clouds,47.6666,-52.7314,Gallows Cove
13,Naze,JP,76.50,overcast clouds,28.3667,129.4833,奄美ポートタワーホテル
15,Khani,GE,76.57,clear sky,41.9563,42.9566,კურორტი ზეკარი/Resort Zekari
16,Bethel,US,73.49,overcast clouds,41.3712,-73.4140,Courtyard by Marriott Danbury


In [45]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name: </dt><dd>{Hotel Name}</dd>
<dt>City: </dt><dd>{City}</dd>
<dt>Country: </dt><dd>{Country}</dd>
<dt>Weather Description: </dt><dd>{Current Description}</dd>
<dt>Max Temp: <dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [52]:
max_temp = clean_hotel_df["Max Temp"]
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
                                dissipating=False, max_intensity=300,
                                point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))